In [1]:
# Подключение библиотек
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
dict_df = {
    'ФИО': [],
    'Специализация': [],
    'Стаж': [],
    'Рейтинг': [],
            } 

In [10]:
for page in range(1,7):
    # Отправляем GET-запрос на сайт продокторов
    url = 'https://prodoctorov.ru/pskov/vrach/?page=' + str(page)
    response = requests.get(url)
   
    # Создаем объект BeautifulSoup для парсинга HTML-кода страницы
    soup = BeautifulSoup(response.text, "lxml")
   
    # Находим элементы с нужной информацией о врачах на странице 
    doctors = soup.find_all('div', class_='b-doctor-card')

    for doctor in doctors:
            dict_df['ФИО'].append(doctor.find('span', class_ ='b-doctor-card__name-surname').get_text(strip=True))
            dict_df['Специализация'].append(doctor.find('div', class_ = 'b-doctor-card__spec').get_text(strip=True))
 
            years = doctor.find('div', class_='b-doctor-card__experience-years')
            if years:
                dict_df['Стаж'].append(years.get_text(strip=True))
            else:
                dict_df['Стаж'].append('-')
            
            dict_df['Рейтинг'].append(doctor.find('div', class_='b-stars-rate__progress')['style'])
    

    Сформируем датафрейм из полученных данных.  

In [11]:
df = pd.DataFrame(dict_df)
df

,ФИО,Специализация,Стаж,Рейтинг
0,Витовский Ярослав Александрович,"Травматолог, \n \n ...",Стаж 29 лет,width: 133px
1,Денисов Максим Викторович,Педиатр,Стаж 17 лет,width: 107px
2,Ханбабаева Альбина Шамиловна,Терапевт,Стаж 12 лет,width: 88px
3,Наумова Дарья Алексеевна,"Гинеколог, \n \n ...",Стаж 10 лет,width: 81px
4,Гусева Елена Владимировна,Терапевт,Стаж 15 лет,width: 80px
...,...,...,...,...
235,Гидраш Игорь Михайлович,"Уролог, \n \n ...",Стаж 26 лет,width: 61px
236,Липич Ирина Павловна,Детский стоматолог,Стаж 15 лет,width: 60px
237,Россиневич Елизавета Алексеевна,"Стоматолог, \n \n ...",Стаж 2 года,width: 60px
238,Войнов Максим Викторович,"Акушер, \n \n ...",Стаж 10 лет,width: 59px


Посмотрим, у скольких врачей пропущены данные о стаже.

In [12]:
df.loc[df['Стаж'] == '-']

,ФИО,Специализация,Стаж,Рейтинг
12,Безносова Ирина Валентиновна,Детский дерматолог,-,width: 54px
105,Щелчков Сергей Александрович,Стоматолог,-,width: 63px
111,Жизневский Андрей Борисович,Стоматолог-хирург,-,width: 61px
132,Безносова Ирина Валентиновна,Детский дерматолог,-,width: 54px
225,Щелчков Сергей Александрович,Стоматолог,-,width: 63px
231,Жизневский Андрей Борисович,Стоматолог-хирург,-,width: 61px


     Отредактируем полученные данные

In [13]:
df['Рейтинг'] = df['Рейтинг'].str.replace(r"[^\d\.]", "", regex=True)

In [14]:
df['Специализация'] = df['Специализация'].str.replace('\n', '')

In [15]:
df['Специализация'] = df['Специализация'].str.replace('  ', '')

    На сайте рейтинг отображается блоком из 5 звезд. Ширина одной звездочки равна 27 px. Для получения рейтинга врача, смотрим ширину блока со звездочками и делим ее на 27. Так узнаем рейтинг врача по шкале от 1 до 5.

In [16]:
df['Рейтинг'] = round(df['Рейтинг'].astype(float) / 27, 2)

    Отредактированные данные отсортируем в порядке убывания рейтинга.

In [17]:
df.sort_values(by='Рейтинг', ascending=False)

,ФИО,Специализация,Стаж,Рейтинг
54,Грищук Мария Петровна,Акушер,Стаж 10 лет,5.00
174,Грищук Мария Петровна,Акушер,Стаж 10 лет,5.00
0,Витовский Ярослав Александрович,"Травматолог, ортопед",Стаж 29 лет,4.93
120,Витовский Ярослав Александрович,"Травматолог, ортопед",Стаж 29 лет,4.93
55,Бевзюк Михаил Игоревич,"Челюстно-лицевой хирург, стоматолог-имплантоло...",Стаж 15 лет,4.89
...,...,...,...,...
31,Подлужная Евгения Юрьевна,Дефектолог,Стаж 6 лет,0.00
30,Гукасова Нина Николаевна,Невролог,Стаж 38 лет,0.00
50,Исправникова Светлана Ивановна,Детский невролог,Стаж 27 лет,0.00
51,Калинина Елена Ивановна,Функциональный диагност,Стаж 26 лет,0.00


    Сохраним полученные данные в csv-файле.

In [18]:
df.to_csv('prodoctorov.csv', sep=';', index = False)